In [103]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [104]:
#timestamp, price, sidem, mideprice, bookfeature, alpha, minprice, maxprice
trade = pd.read_csv('../Data/2018-07-trade-new.csv')

In [105]:
#buy, sell을 한 시점 이후 5분간의 data의 price값들의 min값과 max값을 예측하도록 학습을 한 후에
#예측값과 현재 시점의 가격을 비교해 buy와 sell의 행동을 실행할지 결정한다.

#300개의 data를 train에 사용하고 나머지 229개의 data를 test에 사용한다.
#timestamp는 feature로 사용하지 않기때문에 data는 순차적으로 나눠서 사용하여도 크게 상관없을것이라고 판단했다.
train = trade[:100]
test = trade[100:]
print(train)
print(test)

    Unnamed: 0            timestamp    price  side   midprice   bookfeature  \
0            0  2018-07-01 02:04:44  7109000     0  7097500.0  8.240648e+06   
1            1  2018-07-01 02:05:54  7112000     1  7100000.0  2.816755e+06   
2            2  2018-07-01 02:42:54  7094000     0  7075000.0  8.322289e+06   
3            3  2018-07-01 02:44:44  7094000     1  7076500.0  5.985209e+06   
4            4  2018-07-01 02:46:33  7094000     1  7074500.0  3.317512e+06   
..         ...                  ...      ...   ...        ...           ...   
95          95  2018-07-01 12:58:45  7078000     1  7063500.0  1.668066e+06   
96          96  2018-07-01 12:58:45  7078000     1  7063500.0  1.668066e+06   
97          97  2018-07-01 14:32:02  7084000     0  7073500.0  1.152996e+07   
98          98  2018-07-01 14:34:55  7083000     0  7069500.0  3.126838e+05   
99          99  2018-07-01 14:46:28  7111000     0  7095000.0 -2.266230e+06   

           alpha  minprice  maxprice  
0   1.169760

In [106]:
# min값을 예측하도록 나눈 데이터, feature는 구매 순간 알 수 있는 price, midprice, bookfeature, alpha를 사용한다.
min_train_X = train.filter(['price','midprice','bookfeature','alpha'])
min_train_Y = train.filter(['minprice']).to_numpy().ravel()

#max값을 예측하도록 나눈 데이터,feature는 구매 순간 알 수 있는 price, midprice, bookfeature, alpha를 사용한다.
max_train_X = train.filter(['price','midprice','bookfeature','alpha'])
max_train_Y = train.filter(['maxprice']).to_numpy().ravel()

In [107]:
#학습 data를 이용해 학습
min_reg = LinearRegression().fit(min_train_X, min_train_Y)
max_reg = LinearRegression().fit(max_train_X, max_train_Y)

In [108]:
#min값 예측 정확도를 확인하기 위한 data
min_test_X = test.filter(['price','midprice','bookfeature','alpha'])
min_test_Y = test.filter(['minprice']).to_numpy().ravel()

#max값 예측 정확도를 확인하기 위한 data
max_test_X = test.filter(['price','midprice','bookfeature','alpha'])
max_test_Y = test.filter(['maxprice']).to_numpy().ravel()

In [109]:
#각 예측의 정확도
print("min score : ", min_reg.score(min_test_X, min_test_Y))
print("max score : ", max_reg.score(max_test_X, max_test_Y))

min score :  0.9953983838869879
max score :  0.9793102117692366


In [138]:
#기존 data의 price를 변경할때 참고할 data를 만든다.
test_result = test
test_result['predict_min_price'] = min_reg.predict(min_test_X) 
test_result['predict_max_price'] = max_reg.predict(max_test_X)

#예측값들을 정수형태로 바꾼다.
test_result['predict_min_price'] = test_result['predict_min_price'].apply((lambda x : int(x)))
test_result['predict_max_price'] = test_result['predict_max_price'].apply((lambda x : int(x)))
test_result = test_result.filter(['predict_max_price','predict_min_price','timestamp'])

In [142]:
# 현재 시점 이후 5분 최저값과 최고값을 예측하고 buy나 sell을 그 시점으로 미루어 하는것을 전제로 price값을 해당 price로 변경한다.
# 기존의 data에서 price값을 예측한 값으로 변경하고 Task 1과 같은 방식으로 얼마나 이익을 보는지 확인한다.
# sell의 경우에는 price를 maxprice로 바꾸고 buy의 경우에는 price를 minprice로 바꾼다
trade = pd.read_csv('../Data/2018-07-trade.csv')
for i in range(len(trade)):
    t = trade['timestamp'][i]
    predict_price = test_result[test_result['timestamp'].isin([t])]
    if(len(predict_price)):
        side = trade['side'][i]
        if side == 0:
            price = predict_price['predict_min_price'].min()
            trade['price'][i] = price
            trade['amount'][i] = -(trade['quantity'][i] * price)
        else:
            price = predict_price['predict_max_price'].min()
            trade['price'][i] = price
            trade['amount'][i] = trade['quantity'][i] * price
            
trade.to_csv("../Data/2018-07-trade_predict.csv")

<ipython-input-142-055ec9dacc04>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade['price'][i] = price
<ipython-input-142-055ec9dacc04>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade['amount'][i] = trade['quantity'][i] * price
<ipython-input-142-055ec9dacc04>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trade['price'][i] = price
<ipython-input-142-055ec9dacc04>:13: SettingWithCopyWarning: 
A value is trying to be se

In [143]:
"""
- Taks1 
Compute the total profit of July 1-2 in KRW. It simply means how much money do we make or lose?
The profit calculation moment should be when the accumulative quantity is 0 (only consider 4 digit floating number, ignore the rest). 
"""

print(trade['amount'].groupby(trade['side']).sum())
print(trade['amount'].sum())

side
0    83.912906
1    83.912940
Name: quantity, dtype: float64
side
0   -596814930
1    603112872
Name: amount, dtype: int64
6297942
